In [1]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg itkwidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

from tqdm import tqdm
import itertools
from functools import wraps

c:\ProgramData\Anaconda3\envs\mri_project\lib\site-packages\nilearn\__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [3]:
annotation_boxes = pd.read_excel("../../Simple Path Dataset/Annotation_Boxes.xlsx").set_index("Patient ID")

DATASET_PATH = '../../New Dataset/'
# SEGMENTED_DATASET_PATH = '../../Segmented Dataset/'

In [4]:
annotation_boxes

,Start Row,End Row,Start Column,End Column,Start Slice,End Slice
Patient ID,,,,,,
Breast_MRI_001,234,271,308,341,89,112
Breast_MRI_002,251,294,108,136,59,72
Breast_MRI_003,351,412,82,139,96,108
Breast_MRI_004,262,280,193,204,86,95
Breast_MRI_005,188,213,138,178,76,122
...,...,...,...,...,...,...
Breast_MRI_918,345,395,338,395,62,85
Breast_MRI_919,285,312,369,397,98,109
Breast_MRI_920,172,193,337,355,87,101


In [5]:
annotation_boxes.loc['Breast_MRI_001']

Start Row       234
End Row         271
Start Column    308
End Column      341
Start Slice      89
End Slice       112
Name: Breast_MRI_001, dtype: int64

In [6]:
filenames = ['pre.nii.gz', 'post_1.nii.gz']

In [7]:
# # generate segmentation masks for each patient in the directory

# for item in tqdm(os.listdir(DATASET_PATH)):
#     patient_path = os.path.join(DATASET_PATH, item)

#     x = annotation_boxes.loc[item]
#     row1 = x['Start Row']
#     row2 = x['End Row']

#     col1 = x['Start Column']
#     col2 = x['End Column']

#     slice1 = x['Start Slice']
#     slice2 = x['End Slice']

#     if(os.path.isdir(patient_path)):
#         # find files with paths here
#         nifti_img = nib.load(os.path.join(patient_path, filenames[0]))   #load pre path and generate segments from this
#         mask = np.zeros(nifti_img.dataobj.shape)
#         mask[row1:row2, col1:col2, slice1:slice2] = 1
#         segment = nib.Nifti1Image(mask, affine = nifti_img.affine)
#         nib.save(segment, os.path.join(patient_path, 'segmentation.nii.gz'))

In [8]:
feat_ext = featureextractor.RadiomicsFeatureExtractor()
feat_ext.settings['minimumROIDimensions'] = 2
feat_ext.settings['normalize'] = True
feat_ext.settings['additionalInfo'] = False
feat_ext.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': True,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': False}

In [18]:
def slice_by_slice_decorator(func):
    """
    A function decorator which executes func on each 3D sub-volume and
    *in-place* pastes the results into the input image. The input image type
    and the output image type are required to be the same type.

    :param func: A function which take a SimpleITK Image as it's first
    argument and returns an Image as results.

    :return: A decorated function.
    """

    iter_dim = 2

    @wraps(func)
    def slice_by_slice(image, *args, **kwargs):

        dim = image.GetDimension()

        if dim <= iter_dim:
            #
            image = func(image, *args, **kwargs)
            return image

        extract_size = list(image.GetSize())
        extract_size[iter_dim:] = itertools.repeat(0, dim - iter_dim)

        extract_index = [0] * dim
        paste_idx = [slice(None, None)] * dim

        extractor = sitk.ExtractImageFilter()
        extractor.SetSize(extract_size)

        for high_idx in itertools.product(
            *[range(s) for s in image.GetSize()[iter_dim:]]
        ):

            # The lower 2 elements of extract_index are always 0.
            # The remaining indices are iterated through all indexes.
            extract_index[iter_dim:] = high_idx
            extractor.SetIndex(extract_index)

            # Sliced based indexing for setting image values internally uses
            # the PasteImageFilter executed "in place".  The lower 2 elements
            # are equivalent to ":". For a less general case the assignment
            # could be written as image[:,:,z] = ...
            paste_idx[iter_dim:] = high_idx
            image[paste_idx] = func(extractor.Execute(image), *args, **kwargs)

        return image

    return slice_by_slice

In [21]:
features = pd.DataFrame()

ad_histeq_2d = slice_by_slice_decorator(sitk.AdaptiveHistogramEqualization)

for patient in tqdm(os.listdir(DATASET_PATH)):
    # for filename in filenames:
        # get features for pre and post_1
    file_dir = os.path.join(DATASET_PATH, patient)
    pre_img = sitk.ReadImage(os.path.join(file_dir, 'pre.nii.gz'))
    post_1_img = sitk.ReadImage(os.path.join(file_dir, 'post_1.nii.gz'))

    ad_histeq_2d(pre_img)
    print('done1')

    ad_histeq_2d(post_1_img)
    print('done2')

    difference_arr = post_1_img - pre_img

    break

        # smt.SetOrigin(img.GetOrigin())
        # feats = feat_ext.execute(img, smt)
        # for key, value in feats.items():
        #     feats[key] = value.item()
        # ft_df = pd.DataFrame(feats, index = [0])
        # ft_df['patient'] = patient
        # ft_df['sequence'] = filename.split('.nii.gz')[0]
        # features = pd.concat([features, ft_df])

  0%|          | 0/922 [00:00<?, ?it/s]

done1


  0%|          | 0/922 [00:07<?, ?it/s]

done2


In [14]:
sitk.Show(pre_img)

In [15]:
sitk.Show(post_1_img)

In [17]:
sitk.Show(difference_arr)

In [ ]:
# features.to_csv("pyradiomics_extraction.csv", index = False)

In [ ]:
result

AttributeError: 'Image' object has no attribute 'dataobj'

In [ ]:
import SimpleITK as sitk
import numpy as np

# Create a noise Gaussian blob test image
img = sitk.GaussianSource(sitk.sitkFloat32, size=[240,240,48], mean=[120,120,24])
img = img + sitk.AdditiveGaussianNoise(img,10)

# Create a ramp image of the same size
h = np.arange(0.0, 255,1.0666666666, dtype='f4')
h2 = np.reshape(np.repeat(h, 240*48), (48,240,240))
himg = sitk.GetImageFromArray(h2)
print(himg.GetSize())

# Match the histogram of the Gaussian image with the ramp
result=sitk.HistogramMatching(img, himg)

# Display the 3d image
import itkwidgets
itkwidgets.view(result)

(240, 240, 48)


TraitError: label_image_weights shape expected to have 1 components, but got () components